## This notebook shows how you can use the `fastplotlib` API to link `Graphic` objects together

In [ ]:
import numpy as np
from fastplotlib import GridPlot, Plot
from ipywidgets import IntSlider, VBox, Layout
import tifffile
from fastplotlib.graphics.line_slider import LineSlider

In [ ]:
# load in contours and temporal components as np arrays
contours = np.load("./data/contours.npy", allow_pickle=True)
temporal = np.load("./data/temporal.npy", allow_pickle=True)

# load in Sue demo movie, may need to change paths
input_movie = tifffile.imread("/home/caitlin/caiman_data/example_movies/Sue_2x_3000_40_-46.tif")

In [ ]:
# generating random colors, will be automatic in the future
rand_colors = np.random.rand(len(contours), 4)  # [n_contours, RGBA]
rand_colors[:, -1] = 1 # set alpha = 1

In [ ]:
# create a gridplot to view the contours over the input movie and the temporal components as a stack
grid_plot = GridPlot((1, 2),  
                     names=[
                         ["contours", "temporal"]
                     ])

# can add graphics to subplots by name or position 
movie_graphic = grid_plot["contours"].add_image(data=input_movie[0], cmap="gnuplot2") 
contour_graphic = grid_plot[0,0].add_line_collection(data=contours, colors=rand_colors)
temporal_stack = grid_plot["temporal"].add_line_stack(data=temporal, colors=rand_colors)


# create a plot to individually view the temporal components
plot = Plot()
temporal_graphic = plot.add_line_collection(data=temporal, colors=rand_colors)

# create slider to dynamically move through input movie
slider = IntSlider(min=0, max=input_movie.shape[0] - 1, value=0, step=1)

# vertical line slider to accompany the individual viewing of temporal components over time
_ls = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max()), slider=slider)
plot.add_graphic(_ls)

# vertical line slider to accompany the temporal line stack
_ls2 = LineSlider(x_pos=0, bounds=(temporal.min(), temporal.max() + temporal_stack.graphics[-1].position.y), slider=slider)
grid_plot["temporal"].add_graphic(_ls2)

# function to update each frame
def update_frame(change):
    ix = change["new"]
    movie_graphic.data = input_movie[ix]
    
slider.observe(update_frame, "value")

@plot.renderer.add_event_handler("resize")
# adds an event handler to resize the temporal line being shown
def update_slider_width(*args):
    # gets the logical size of the plot graphic
    width, h = plot.renderer.logical_size
    # sizes the vertical slider to fit width of temporal component 
    slider.layout = Layout(width=f"{width}px")

VBox([grid_plot.show(), plot.show(), slider])

### autoscaling

In [ ]:
plot.auto_scale()
plot.camera.scale.x = 0.85
grid_plot["temporal"].auto_scale()

In [ ]:
# not important to understand for now, will be made public in the next version
def euclidean(source, target, event, new_data):
    """maps click events to contour"""
    # calculate coms of line collection
    indices = np.array(event.pick_info["index"])
    
    coms = list()

    for contour in target.graphics:
        coors = contour.data()[~np.isnan(contour.data()).any(axis=1)]
        com = coors.mean(axis=0)
        coms.append(com)

    # euclidean distance to find closest index of com 
    indices = np.append(indices, [0])
    
    ix = int(np.linalg.norm((coms - indices), axis=1).argsort()[0])
    
    # should typically never access private methods
    target._set_feature(feature="colors", new_data=new_data, indices=ix)
    
    return None

### interactivity

In [ ]:
# so we can view the temporal graphics one-by-one, first hide all of them
# this will possible be incorporated into link() in the future
temporal_graphic[:].present = False

# link image to contours
# multiselection is also coming in the near future
movie_graphic.link(
    "click",
    target=contour_graphic,
    feature="colors", 
    new_data="w", 
    callback=euclidean
)

# contours colors -> contour thickness
contour_graphic.link("colors", target=contour_graphic, feature="thickness", new_data=5)

# temporal stack events 
temporal_stack.link("click", target=temporal_stack, feature="colors", new_data="w")
temporal_stack.link("colors", target=temporal_stack, feature="thickness", new_data=4)

# contours <-> temporal stack
contour_graphic.link("colors", target=temporal_stack, feature="colors", new_data="w", bidirectional=True)

# temporal stack -> temporal single
temporal_stack.link("colors", target=temporal_graphic, feature="present", new_data=True)

# autoscale temporal plot to the current temporal component
temporal_graphic[:].present.add_event_handler(plot.auto_scale)

In [ ]:
# make sure to close plots and kill kernel
plot.canvas.close()
grid_plot.canvas.close()

# API Stability 

## Little Changes
- `PlotArea` and its subclasses (`Subplot`, `Plot`, etc.)
- `Graphics`: their use and definition 
- `Gridplot`

## Moderate Changes
- Interaction
- Graphic features
- Graphic collections
- Graphic & graphic subclasses
- `ImageWidget`

## Heavy Changes
- `Heatmap` graphic
- `Histogram` graphic
- `LineSlider` graphic


# Ways to contribute:

### 1. general `Theme` class (handling background color, gridplot spacing, etc.)

### 2. creating a `rectangle` graphic

### 3. different sizes for `scatter` graphic

#### Can also see _Roadmap 2023_ for more ways to contribute!

## If you are feeling overwhelmed, do NOT worry! There will be more `fastplotlib` usage in later sessions today to get more practice.